# Data Preprocessing

In [47]:
import pandas as pd

In [48]:
# german_file_path="Data_Sets/deu.txt"
# spanish_file_path="Data_Sets/data.csv"

In [49]:
# Initialize lists to store German and English data
# german_data = []
# english_data = []

# with open(german_file_path,'r',encoding='utf-8') as file:
#     for line in file:
#         english,german,_=line.strip().split('\t')
#         english_data.append(english)
#         german_data.append(german)
        

# # Create a DataFrame using Pandas
# df = pd.DataFrame({ 'English': english_data , 'German': german_data})

# # Display the first few rows of the DataFrame
# print(df[:20])        

In [50]:
# df_spanish=pd.read_csv(spanish_file_path)

# # renaming the names
# new_column_names={'english':'English','spanish':'Spanish'}
# df_spanish=df_spanish.rename(columns=new_column_names)

# print(df_spanish[:10])

In [51]:
# file_name=f"data_german.csv"

# df.to_csv(f"Data_Sets/{file_name}",index=False)
# print("DataFrame saved successfully as", file_name)

In [52]:
import os
# current_file_path="Data_Sets/data.csv"
# new_file_path="Data_Sets/data_spanish.csv"

# os.rename(current_file_path,new_file_path)

# print("File renamed successfully to", new_file_path)

In [53]:
import pandas as pd
german_file_path="Data_Sets/data_german.csv"
spanish_file_path="Data_Sets/data_spanish.csv"

In [54]:
df1=pd.read_csv(german_file_path)
df2=pd.read_csv(spanish_file_path)

In [55]:
import random
import tensorflow as tf
import string
import re
import csv
from tensorflow import keras
from tensorflow.keras import layers

In [56]:
df1.head(20)

,English,German
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!
5,Wow!,Potzdonner!
6,Wow!,Donnerwetter!
7,Fire!,Feuer!
8,Help!,Hilfe!
9,Help!,Zu Hülf!


In [72]:
first_column_values_last_10_rows=df1['English'].tail(10)


paragraph = '\n'.join(first_column_values_last_10_rows)
# Print the paragraph
print(paragraph)

As a prank, some students let three goats loose inside their school after painting the numbers 1, 2 and 4 on the sides of the goats. The teachers spent most of the day looking for goat number 3.
The small crowd at Hiroshima Peace Memorial Park stood for a moment of silence at 8:15 a.m., the exact moment an atomic bomb nicknamed “Little Boy” was dropped from the U.S. warplane Enola Gay.
In today's world, we have to equip all our kids with an education that prepares them for success, regardless of what they look like, or how much their parents make, or the zip code that they live in.
Death is something that we're often discouraged to talk about or even think about, but I've realized that preparing for death is one of the most empowering things you can do. Thinking about death clarifies your life.
At a moment when our economy is growing, our businesses are creating jobs at the fastest pace since the 1990s, and wages are starting to rise again, we have to make some choices about the kind o

In [73]:
len(df1)

221533

# Split the English and german translation pairs

In [74]:
#common function for both spanish and german

class split_pairs:
 def split_pairs_method(self,df1): 
  text_pairs=[]
  i=0
  for i in range(len(df1)):
    english,language=df1["English"][i],df1["German"][i]
    german="[start] "+language+" [end]"
    text_pairs.append((english,language))

  return text_pairs    


In [75]:
#randomly selecting that if the above function work
class random_pair_test:
  def random_test_method(self,text_pairs): 
   for i in range(3):
    print(random.choice(text_pairs))
    

In [76]:
#pairing for german text

german_text_pairs=split_pairs().split_pairs_method(df1)

random_pair_test().random_test_method(german_text_pairs)

("Tom is going to stay, isn't he?", 'Tom wird bleiben, oder?')
('The matter is really pressing.', 'Die Angelegenheit ist von großer Dringlichkeit.')
('I will do anything that you ask.', 'Ich werde alles tun, was du verlangst.')


# Randomizing the data

In [77]:
random.shuffle(german_text_pairs)

# Split the data into training, validation,testing

In [63]:
#class for splitting text pairs in to train,test,validation
class splitting:
    def splitting_method(self,text_pairs):
        num_val_sample=int(0.15*len(text_pairs))
        num_train_samples=len(text_pairs) - 2 * num_val_sample
        train_pairs=text_pairs[:num_train_samples]
        val_pairs=text_pairs[num_train_samples:num_train_samples+num_val_sample]
        test_pairs=text_pairs[num_train_samples+num_val_sample:]
        
        print("Total Sentences: ",len(text_pairs))
        print("Training set size: ",len(train_pairs))
        print("Validation set size: ",len(val_pairs))
        print("Testng set size: ",len(test_pairs))
        return train_pairs,val_pairs,test_pairs    



In [64]:
german_train_pairs,german_val_pairs,german_test_pairs=splitting().splitting_method(german_text_pairs)


Total Sentences:  221533
Training set size:  155075
Validation set size:  33229
Testng set size:  33229


In [65]:
print(len(german_test_pairs)+len(german_train_pairs)+len(german_val_pairs))

221533


In [66]:
print(german_val_pairs[200])

("Tom didn't see anything.", 'Tom hat nichts gesehen.')


# Removing Puncuations

In [67]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


f"[{re.escape(strip_chars)}]"

f"{5+3}"

'8'

# Vectorizing the English and spanish text pairs

In [78]:
def custom_standardization(input_string):
    lowercase=tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase,f"[{re.escape(strip_chars)}]", "")

vocab_size=15000
sequence_length=20

source_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length+1,
    standardize=custom_standardization,
)

train_english_texts=[pair[0] for pair in german_train_pairs]
train_german_texts=[pair[1] for pair in german_train_pairs]

source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_german_texts)

In [79]:
print(train_english_texts[1])
print(train_german_texts[1])

print(source_vectorization)

He was acquitted.
Er ist freigesprochen worden.
<TextVectorization name=text_vectorization_4, built=False>


In [80]:
print(german_train_pairs[1:5])

[('He was acquitted.', 'Er ist freigesprochen worden.'), ('What time does the first train leave?', 'Wann fährt der erste Zug ab?'), ('Health is an important factor of happiness.', 'Gesundheit ist ein wesentlicher Faktor zum Glück.'), ('Tom is on the roof.', 'Tom ist auf dem Dach.')]


# Preparing datasets for the translation task

In [81]:
batch_size=64

def format_dataset(eng,ger):
    eng=source_vectorization(eng)
    ger=target_vectorization(ger)
    return ({"english":eng,
             "german":ger[:,:-1],
             },ger[:,1:])

def make_dataset(pairs):
    eng_texts,ger_texts=zip(*pairs)
    eng_texts=list(eng_texts)
    ger_texts=list(ger_texts)
    dataset=tf.data.Dataset.from_tensor_slices((eng_texts,ger_texts))
    dataset=dataset.batch(batch_size)
    dataset=dataset.map(format_dataset,num_parallel_calls=4)
    
    return dataset.shuffle(2048).prefetch(16).cache()

train_data=make_dataset(german_train_pairs)
val_data=make_dataset(german_val_pairs)

# print(train_data)
# print(val_data)

for inputs, targets in train_data.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['german'].shape: {inputs['german'].shape}")
    print(f"targets.shape: {targets.shape}")

     

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'german': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>
<CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'german': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>
inputs['english'].shape: (64, 20)
inputs['german'].shape: (64, 20)
targets.shape: (64, 20)


In [82]:
print(list(train_data.as_numpy_iterator())[50])

({'english': array([[   6,   18,    7, ...,    0,    0,    0],
       [ 169,   30,   25, ...,    0,    0,    0],
       [  40,    4,  559, ...,    0,    0,    0],
       ...,
       [   2,    8, 1462, ...,    0,    0,    0],
       [ 359,   36,  238, ...,    0,    0,    0],
       [  28,   23,  801, ...,    0,    0,    0]], dtype=int64), 'german': array([[   2,  574,   75, ...,    0,    0,    0],
       [   1,   47,   11, ...,    0,    0,    0],
       [  14,  153, 1122, ...,    0,    0,    0],
       ...,
       [   3,    5,  370, ...,    0,    0,    0],
       [   8,   35,    4, ...,    0,    0,    0],
       [  18,   35,   93, ...,    0,    0,    0]], dtype=int64)}, array([[  574,    75,    39, ...,     0,     0,     0],
       [   47,    11, 13287, ...,     0,     0,     0],
       [  153,  1122,    47, ...,     0,     0,     0],
       ...,
       [    5,   370,  9750, ...,     0,     0,     0],
       [   35,     4,   285, ...,     0,     0,     0],
       [   35,    93,   519, .

# Transformers encoder implemented as a subclassed Layer

In [84]:
class TransformerEncoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention=layers.MultiHeadAttention(
           num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=keras.Sequential(
            [layers.Dense(dense_dim,activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()

    def call(self,inputs,mask=None):
        if mask is not None:
            mask=mask[:,tf.newaxis,:]
        attention_output=self.attention(
            inputs,inputs,attention_mask=mask
        )
        project_input=self.layernorm_1(inputs+attention_output)
        project_output=self.dense_proj(project_input)

        return self.layernorm_2(project_input+project_output)

    def get_config(self):
        config=super().get_config()
        config.spdate({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config        

# Transformer Encorder implemented as a subclassed Layer